<a href="https://colab.research.google.com/github/zombiebeta0/git-test/blob/master/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_Epic_RPG_Auto_Event.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
print('Setting up...')
!pip install discord
import discord
import asyncio
from datetime import datetime
from random import randint, random
from threading import Timer
import sys
import nest_asyncio
nest_asyncio.apply()

from google.colab import output
output.clear()

In [ ]:
#@title Auto Epic RPG Grinder
#@markdown Account
TOKEN = '' #@param {type: "string"}
#@markdown Events
Join = True #@param {type: "boolean"}
Catch = False #@param {type: "boolean"}
Fish = True #@param {type: "boolean"}
Chop = True #@param {type: "boolean"}
TimeToFight = True #@param {type: "boolean"}
waiting_time = 3.5 #@param {type: "number"}

switches = {
    'join': Join,
    'catch': Catch,
    'fish': Fish,
    'chop': Chop,
    'time to fight': TimeToFight
}


client = discord.Client()

react_messages = {
    'join': "Type `join` to join the arena!",
    'catch': "IT'S RAINING COINS",
    'fish': "A MEGALODON HAS SPAWNED",
    'chop': "AN EPIC TREE HAS JUST GROWN",
    'time to fight': "A LEGENDARY BOSS JUST SPAWNED"
}
room = {}
start_time = datetime.now()
stats = {
    react: {'success': 0,
            'total': 0,
            'gain': 0} for react in react_messages
}
units = {
    'join': 'cookies',
    'fight': 'wins',
    'catch': 'coins',
    'fish': 'fish',
    'chop': 'logs',
    'time to fight': 'levels'
}
react_lists = ['cookies', 'arena', 'join', 'rest easy',
                'coin', 'catch', "cath'em",
                'fish', 'megalo',
                'chop', 'epic tree', 'bone',
                ':shield:', 'legendary boss', 'strongest', 'time to fight']
queue = []
count = 0
pause = False

def get_time():
    return str(datetime.now().time())[:11]

async def main_loop():
    global count
    while True:
        if not pause and len(queue) > 0:
            channel, text = queue.pop()
            await channel.send(text)
            await asyncio.sleep(1)
            count +=1
            if count % 20 == 0:
                output.clear()
        await asyncio.sleep(0.1)

@client.event
async def on_message(message):
    global switches, waiting_time, queue, pause
    author = message.author
    content = message.clean_content
    if message.guild:
        if author == client.user:
            if content.startswith('>on'):
                switches[content[4:]] = True
            elif content.startswith('>off'):
                switches[content[5:]] = False
            elif content.startswith('>wait'):
                waiting_time = float(content[5:])
            elif content == '>stat':
                diff = (datetime.now() - start_time).total_seconds()
                print('*' * 90)
                print(get_time(), '\t', int(diff//3600), 'hours', int(diff%3600//60), 'mins', int(diff%60), 'secs from', str(start_time.time())[:11])
                for react in stats:
                    stat = stats[react]
                    print()
                    print('\t', react.upper() if react != 'time to fight' else 'BOSS', '\t', stat['success'], '/', stat['total'], f"({round(stat['success'] / stat['total'] * 100, 2) if stat['total'] else '0.00'} %)", '\t',
                            '+', stat['gain'], units[react], f"({int(stat['gain'] / diff * 60)} per min)")
                print('*' * 90)
            elif content == '>pause' or content == '>pause event':
                queue = []
                pause = True
            elif content == '>resume' or content == '>resume event':
                pause = False
            elif content == '>exit' or content == '>exit event':
                sys.exit()
        elif message.guild.name == "Gamer's Club":
            return
    if pause:
        return

    chid = message.channel.id
    embed = message.embeds[0] if len(message.embeds) > 0 else None
    field = embed.fields[0] if embed and len(embed.fields) > 0 else None
    
    if author.name == 'EPIC RPG' and field:
        name = field.name
        if 'Everyone got' in name and (('fish' in name and switches['fish']) or \
                                        ('logs' in name and switches['chop']) or \
                                        ('coins' in name and switches['catch'])):
            print('-' * 90)
            print(get_time(), '\t', message.guild, '\t', message.channel)
            room.pop(chid, None)
            players = embed.description[9:].split(', ')
            react = 'fish' if 'fish' in name else 'chop' if 'logs' in name else 'catch'
            text = name[name.find('Everyone'):]
            stats[react]['total'] += 1
            print('\t\t', text, end='\t')
            if client.user.name in players:
                stats[react]['success'] += 1
                stats[react]['gain'] += int(''.join(text.split()[2].split(',')))
                print('You are', players.index(client.user.name) + 1, '/', len(players))
            else:
                print('You missed it')
            print('-' * 90)
        elif 'THE LEGENDARY BOSS DIED! EVERYONE LEVELED UP' in name and switches['time to fight']:
            print('-' * 90)
            print(get_time(), '\t', message.guild, '\t', message.channel)
            room.pop(chid, None)
            players = field.value[9:].split(', ')
            react = 'time to fight'
            stats[react]['total'] += 1
            print('\t\t', 'THE LEGENDARY BOSS DIED! EVERYONE LEVELED UP', end='\t')
            if client.user.name in players:
                stats[react]['success'] += 1
                stats[react]['gain'] += 1
                print('You are', players.index(client.user.name) + 1, '/', len(players))
            else:
                print('You missed it')
            print('-' * 90)
        elif 'The legendary boss has not been defeated' in name and switches['time to fight']:
            print('-' * 90)
            print(get_time(), '\t', message.guild, '\t', message.channel)
            room.pop(chid, None)
            print('\t\t', "'The legendary boss has not been defeated")
            print('-' * 90)
        elif 'Ranking' in name and switches['join']:
            print('-' * 90)
            print(get_time(), '\t', message.guild, '\t', message.channel)
            room.pop(chid, None)
            players = embed.description[9:].split(', ')
            react = 'join'
            stats[react]['total'] += 1
            print('\t\t Arena Result', end='\t')
            if client.user.name in players:
                idx = field.value.find(client.user.name + ' (')
                cookie = field.value[idx + 4: field.value.find(' <:', idx)]
                stats[react]['success'] += 1
                stats[react]['gain'] += int(cookie[1:])
                print('You are', players.index(client.user.name) + 1, '/', len(players), '\t', cookie, 'cookies')
            else:
                print('You missed it')
            print('-' * 90)
        else:
            for react in react_messages:
                if switches[react] and react_messages[react] in name:
                    room[chid] = [set(), react, True]
                    print(get_time(), '\t', message.guild, '\t', message.channel, f'\t{react.upper()} EVENT')
                    break
    elif chid in room:
        group, react, usable = room[chid]
        if not switches[react]:
            return
        content_lower = content.lower()
        if author.bot:
            if author.name != 'EPIC RPG' and usable:
                if any([word in content_lower for word in react_lists]):
                    room[chid][2] = False
                    if message.channel.name == 'tricks-or-treats':
                        t = waiting_time - 1
                    elif message.channel.name.startswith('epic-event-drop'):
                        t = waiting_time - 1
                    elif message.channel.name == 'arena-miniboss':
                        t = waiting_time - 1.5
                    else:
                        t = waiting_time
                    print(get_time(), '\t', message.guild, '\t', message.channel, '\twaiting for %.2f s to' % t, react.upper())
                    await asyncio.sleep(t)
                    print(get_time(), '\t', message.guild, '\t', message.channel, f'\t{react.upper()}')
                    queue.append((message.channel, react))
        elif author.name not in group:
            group.add(author.name)
            print(get_time(), '\t', message.guild, '\t', message.channel, f'\t{len(group)} {author.name}')
            if len(group) >= 20:
                room.pop(chid, None)

@client.event
async def on_ready():
    print('Logged in as', client.user)
    print('-' * 90)

    client.loop.create_task(main_loop())

client.run(TOKEN, bot=False)

------------------------------------------------------------------------------------------
01:15:21.69 	 EPIC REALM 	 miniboss-arena-only
		 Arena Result	You missed it
------------------------------------------------------------------------------------------
Logged in as Po#1004
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
01:15:22.99 	 EPIC RPG Gladiators 	 miniboss-and-arena
		 Arena Result	You missed it
------------------------------------------------------------------------------------------
01:15:30.42 	 RPG ARMY 	 🍪╏arena-miniboss 	JOIN EVENT
01:15:30.49 	 RPG ARMY 	 🍪╏arena-miniboss 	waiting for 3.50 s to JOIN
01:15:33.53 	 RPG ARMY 	 🍪╏arena-miniboss 	1 Henry🤙
01:15:33.79 	 RPG ARMY 	 🍪╏arena-miniboss 	2 Henri
01:15:34.00 	 RPG ARMY 	 🍪╏arena-miniboss 	JOIN
01:15:34.21 	 RPG ARMY 	 🍪╏arena-miniboss 	3 Po
01:15:35.18 	 RPG ARMY 	 🍪╏arena-minibo

RuntimeError: ignored